In [15]:
# %load trainer.py
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
import torch
import torchsummary

In [ ]:

DOWNLOAD_MNIST = False   #第一次用要先下載data,所以是True

# prepare dataset

transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])
train_data = torchvision.datasets.MNIST(
    root='./mnist',
    train=True,
    transform=transform, 
    download=DOWNLOAD_MNIST
)

test_data = torchvision.datasets.MNIST(
    root='./mnist/', 
    train=False,
    transform=transform,
    download=DOWNLOAD_MNIST,
)

In [ ]:

# # Check dataset shape
# train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=1, shuffle=True)

# # Visualize one resized image
# example_image, example_label = next(iter(train_loader))
# # print(f"Image shape: {example_image.shape}")  # Should be (1, 1, 32, 32)

# print(test_data.test_data.size())





torch.Size([10000, 28, 28])


/Users/yiting/Course/cv2024/env/lib/python3.12/site-packages/torchvision/datasets/mnist.py:81: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


In [ ]:
import torchvision
import torch.nn as nn
import torchsummary

num_classes = 10

# 原始 VGG16 模型
model = torchvision.models.vgg16(pretrained=True)
for param in model.features.parameters():
    param.requires_grad = False
# 修改輸入通道數
model.features[0] = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

# 插入 Batch Normalization
new_features = []
for layer in model.features:
    new_features.append(layer)
    if isinstance(layer, nn.Conv2d):  # 在每個卷積層後加 BN
        new_features.append(nn.BatchNorm2d(layer.out_channels))

# 替換原有的 features 模組
model.features = nn.Sequential(*new_features)

# 修改輸出層
model.classifier._modules['6'] = nn.Linear(model.classifier[6].in_features, num_classes)

# 查看模型結構
torchsummary.summary(model, (1, 32, 32))

KeyError: 'imagenet'

In [17]:
#set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cpu


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

EPOCH = 40
lr = 0.001

# Loss and optimizer
criterion = nn.CrossEntropyLoss(reduction='mean')
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Initialize lists to save loss and accuracy
train_losses = []
test_accuracies = []

# Variable to track the best accuracy
best_accuracy = 0.0

# Train the model
for epoch in range(EPOCH):
    model.train()  # Set model to training mode
    train_loss = 0.0
    total_train = 0
    correct_train = 0

    for images, labels in train_loader:
        # No need for Variable, tensors work directly
        images, labels = images.to(device), labels.to(device)  # Move to GPU if available

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        train_loss += loss.item() * images.size(0)  # Accumulate batch loss

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate training accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    # Calculate average training loss and accuracy
    train_loss /= len(train_loader.dataset)
    train_accuracy = 100 * correct_train / total_train

    # Evaluate the model
    model.eval()  # Set model to evaluation mode
    correct_test = 0
    total_test = 0

    with torch.no_grad():  # Disable gradient computation for testing
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Move to GPU if available

            # Forward pass
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    test_accuracy = 100 * correct_test / total_test

    # Output training and testing results
    print(f"Epoch [{epoch+1}/{EPOCH}]")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")
    print(f"Test Accuracy: {test_accuracy:.2f}%\n")

    # Save losses and accuracies
    train_losses.append(train_loss)
    test_accuracies.append(test_accuracy)

    # Save the best model based on test accuracy
    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy
        torch.save(model.state_dict(), 'best_model.ckpt')
        print(f"Best model saved with accuracy: {best_accuracy:.2f}%")

# Print final results
print("Training Losses:", train_losses)
print("Test Accuracies:", test_accuracies)